<a href="https://colab.research.google.com/github/kcanmersin/Google-Colab-Notebooks/blob/main/fifa_man_of_the_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [435]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
import os
import kaggle

def download_kaggle_dataset_to_drive(dataset_name, drive_folder_path):
    # Create a folder in Google Drive to store the downloaded dataset
    if not os.path.exists(drive_folder_path):
        os.makedirs(drive_folder_path)

    # Set the path where you want to download the dataset in the drive_folder_path
    download_path = os.path.join(drive_folder_path, dataset_name)

    # Use the Kaggle API to download the dataset files
    kaggle.api.dataset_download_files(dataset_name, path=download_path, unzip=True, quiet=True)

    # List the downloaded files
    downloaded_files = os.listdir(download_path)
    print(f'Downloaded {len(downloaded_files)} files from the dataset: {dataset_name}')

def start(dataset_name):
    # Create a folder in Google Drive to store the downloaded dataset
    drive_folder_path = '/content/drive/MyDrive/KaggleDatasets'
    if not os.path.exists(drive_folder_path):
        os.makedirs(drive_folder_path)

    # Set the path where you want to download the dataset in the drive_folder_path
    download_path = os.path.join(drive_folder_path, dataset_name)

    # Use the Kaggle API to download the dataset files
    kaggle.api.dataset_download_files(dataset_name, path=download_path, unzip=True, quiet=True)

    # List the downloaded files
    downloaded_files = os.listdir(download_path)
    print(f'Downloaded {len(downloaded_files)} files from the dataset: {dataset_name}')

    # Call the separate function to download to Google Drive
    download_kaggle_dataset_to_drive(dataset_name, drive_folder_path)

    # Set the path to the downloaded dataset folder
    csv_file_path = os.path.join(drive_folder_path, dataset_name)
    return csv_file_path

# Example usage:
path = start("mathan/fifa-2018-match-statistics")
print(path)


mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory
Downloaded 1 files from the dataset: mathan/fifa-2018-match-statistics
Downloaded 1 files from the dataset: mathan/fifa-2018-match-statistics
/content/drive/MyDrive/KaggleDatasets/mathan/fifa-2018-match-statistics


In [436]:
csv_file_path = start ('mathan/fifa-2018-match-statistics')

Downloaded 1 files from the dataset: mathan/fifa-2018-match-statistics
Downloaded 1 files from the dataset: mathan/fifa-2018-match-statistics


In [437]:
csv_file_path

'/content/drive/MyDrive/KaggleDatasets/mathan/fifa-2018-match-statistics'

In [438]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
import tensorflow as tf

In [439]:
data = pd.read_csv("/content/drive/MyDrive/KaggleDatasets/mathan/fifa-2018-match-statistics/FIFA 2018 Statistics.csv")

In [440]:
data.head()

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,...,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time
0,14-06-2018,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,0,0,0,Yes,12.0,Group Stage,No,0,NaN,NaN
1,14-06-2018,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,0,0,0,No,NaN,Group Stage,No,0,NaN,NaN
2,15-06-2018,Egypt,Uruguay,0,43,8,3,3,2,0,...,2,0,0,No,NaN,Group Stage,No,0,NaN,NaN
3,15-06-2018,Uruguay,Egypt,1,57,14,4,6,4,5,...,0,0,0,Yes,89.0,Group Stage,No,0,NaN,NaN
4,15-06-2018,Morocco,Iran,0,64,13,3,6,4,5,...,1,0,0,No,NaN,Group Stage,No,0,1.0,90.0


In [441]:
data.columns

Index(['Date', 'Team', 'Opponent', 'Goal Scored', 'Ball Possession %',
       'Attempts', 'On-Target', 'Off-Target', 'Blocked', 'Corners', 'Offsides',
       'Free Kicks', 'Saves', 'Pass Accuracy %', 'Passes',
       'Distance Covered (Kms)', 'Fouls Committed', 'Yellow Card',
       'Yellow & Red', 'Red', 'Man of the Match', '1st Goal', 'Round', 'PSO',
       'Goals in PSO', 'Own goals', 'Own goal Time'],
      dtype='object')

#preprocessing

In [442]:
data.isnull().sum()

Date                        0
Team                        0
Opponent                    0
Goal Scored                 0
Ball Possession %           0
Attempts                    0
On-Target                   0
Off-Target                  0
Blocked                     0
Corners                     0
Offsides                    0
Free Kicks                  0
Saves                       0
Pass Accuracy %             0
Passes                      0
Distance Covered (Kms)      0
Fouls Committed             0
Yellow Card                 0
Yellow & Red                0
Red                         0
Man of the Match            0
1st Goal                   34
Round                       0
PSO                         0
Goals in PSO                0
Own goals                 116
Own goal Time             116
dtype: int64

In [443]:
data.describe()

,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,...,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,1st Goal,Goals in PSO,Own goals,Own goal Time
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,...,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,94.000000,128.000000,12.0,12.000000
mean,1.320312,49.992188,12.593750,3.914062,5.273438,3.359375,4.718750,1.343750,14.890625,2.726562,...,462.648438,106.664062,13.546875,1.695312,0.015625,0.015625,39.457447,0.203125,1.0,45.833333
std,1.156519,10.444074,5.245827,2.234403,2.409675,2.403195,2.446072,1.193404,4.724262,2.049447,...,151.186311,11.749537,4.619131,1.325454,0.124507,0.124507,24.496506,0.807049,0.0,29.978275
min,0.000000,25.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,5.000000,0.000000,...,189.000000,80.000000,5.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,12.000000
25%,0.000000,42.000000,9.000000,2.000000,4.000000,1.750000,3.000000,0.000000,11.000000,1.000000,...,351.000000,101.000000,10.000000,1.000000,0.000000,0.000000,18.250000,0.000000,1.0,21.750000
50%,1.000000,50.000000,12.000000,3.500000,5.000000,3.000000,5.000000,1.000000,15.000000,2.000000,...,462.000000,104.500000,13.000000,2.000000,0.000000,0.000000,39.000000,0.000000,1.0,35.000000
75%,2.000000,58.000000,15.000000,5.000000,7.000000,4.000000,6.000000,2.000000,18.000000,4.000000,...,555.250000,109.000000,16.000000,2.000000,0.000000,0.000000,54.750000,0.000000,1.0,75.750000
max,6.000000,75.000000,26.000000,12.000000,11.000000,10.000000,11.000000,5.000000,26.000000,9.000000,...,1137.000000,148.000000,25.000000,6.000000,1.000000,1.000000,90.000000,4.000000,1.0,90.000000


In [444]:
data.drop(["Own goals","Own goal Time","Date"],axis=1,inplace=True)

In [445]:
data["1st Goal"]= data["1st Goal"].fillna(data["1st Goal"].mean())

In [446]:
data.isnull().sum()

Team                      0
Opponent                  0
Goal Scored               0
Ball Possession %         0
Attempts                  0
On-Target                 0
Off-Target                0
Blocked                   0
Corners                   0
Offsides                  0
Free Kicks                0
Saves                     0
Pass Accuracy %           0
Passes                    0
Distance Covered (Kms)    0
Fouls Committed           0
Yellow Card               0
Yellow & Red              0
Red                       0
Man of the Match          0
1st Goal                  0
Round                     0
PSO                       0
Goals in PSO              0
dtype: int64

#Encode

In [447]:
data.dtypes

Team                       object
Opponent                   object
Goal Scored                 int64
Ball Possession %           int64
Attempts                    int64
On-Target                   int64
Off-Target                  int64
Blocked                     int64
Corners                     int64
Offsides                    int64
Free Kicks                  int64
Saves                       int64
Pass Accuracy %             int64
Passes                      int64
Distance Covered (Kms)      int64
Fouls Committed             int64
Yellow Card                 int64
Yellow & Red                int64
Red                         int64
Man of the Match           object
1st Goal                  float64
Round                      object
PSO                        object
Goals in PSO                int64
dtype: object

In [448]:
data.nunique()

Team                       32
Opponent                   32
Goal Scored                 7
Ball Possession %          43
Attempts                   24
On-Target                  12
Off-Target                 11
Blocked                    11
Corners                    12
Offsides                    6
Free Kicks                 21
Saves                      10
Pass Accuracy %            25
Passes                    110
Distance Covered (Kms)     35
Fouls Committed            21
Yellow Card                 7
Yellow & Red                2
Red                         2
Man of the Match            2
1st Goal                   57
Round                       6
PSO                         2
Goals in PSO                4
dtype: int64

In [449]:
import pandas as pd

# Assuming you have a DataFrame called "data"
for col in data.columns:
    if data[col].dtype == "object":
        print(f"{col}: {data[col].unique()}\n")


Team: ['Russia' 'Saudi Arabia' 'Egypt' 'Uruguay' 'Morocco' 'Iran' 'Portugal'
 'Spain' 'France' 'Australia' 'Argentina' 'Iceland' 'Peru' 'Denmark'
 'Croatia' 'Nigeria' 'Costa Rica' 'Serbia' 'Germany' 'Mexico' 'Brazil'
 'Switzerland' 'Sweden' 'Korea Republic' 'Belgium' 'Panama' 'Tunisia'
 'England' 'Colombia' 'Japan' 'Poland' 'Senegal']

Opponent: ['Saudi Arabia' 'Russia' 'Uruguay' 'Egypt' 'Iran' 'Morocco' 'Spain'
 'Portugal' 'Australia' 'France' 'Iceland' 'Argentina' 'Denmark' 'Peru'
 'Nigeria' 'Croatia' 'Serbia' 'Costa Rica' 'Mexico' 'Germany'
 'Switzerland' 'Brazil' 'Korea Republic' 'Sweden' 'Panama' 'Belgium'
 'England' 'Tunisia' 'Japan' 'Colombia' 'Senegal' 'Poland']

Man of the Match: ['Yes' 'No']

Round: ['Group Stage' 'Round of 16' 'Quarter Finals' 'Semi- Finals' '3rd Place'
 'Final']

PSO: ['No' 'Yes']



In [450]:
label_encoder = LabelEncoder()

data['Man of the Match'] = label_encoder.fit_transform(data['Man of the Match'])
man_mappings = {index: label for index, label in enumerate(label_encoder.classes_)}

data['PSO'] = label_encoder.fit_transform(data['PSO'])
pso_mappings = {index: label for index, label in enumerate(label_encoder.classes_)}

In [451]:
data

,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,...,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO
0,Russia,Saudi Arabia,5,40,13,7,3,3,6,3,...,118,22,0,0,0,1,12.000000,Group Stage,0,0
1,Saudi Arabia,Russia,0,60,6,0,3,3,2,1,...,105,10,0,0,0,0,39.457447,Group Stage,0,0
2,Egypt,Uruguay,0,43,8,3,3,2,0,1,...,112,12,2,0,0,0,39.457447,Group Stage,0,0
3,Uruguay,Egypt,1,57,14,4,6,4,5,1,...,111,6,0,0,0,1,89.000000,Group Stage,0,0
4,Morocco,Iran,0,64,13,3,6,4,5,0,...,101,22,1,0,0,0,39.457447,Group Stage,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,England,Croatia,1,46,11,1,6,4,4,3,...,148,14,1,0,0,0,5.000000,Semi- Finals,0,0
124,Belgium,England,2,43,12,4,3,5,4,1,...,108,11,1,0,0,1,4.000000,3rd Place,0,0
125,England,Belgium,0,57,15,5,7,3,5,0,...,110,5,2,0,0,0,39.457447,3rd Place,0,0
126,France,Croatia,4,39,8,6,1,1,2,1,...,99,14,2,0,0,1,18.000000,Final,0,0


In [452]:
round_values = list(data['Round'].unique())
round_values

['Group Stage',
 'Round of 16',
 'Quarter Finals',
 'Semi- Finals',
 '3rd Place',
 'Final']

In [453]:
round_mappings = {label: index for index, label in enumerate(round_values)}
round_mappings

{'Group Stage': 0,
 'Round of 16': 1,
 'Quarter Finals': 2,
 'Semi- Finals': 3,
 '3rd Place': 4,
 'Final': 5}

In [454]:
data['Round'] = data['Round'].apply(lambda x: round_mappings[x])

In [455]:
data = pd.get_dummies(data, columns=["Team"])
data = pd.get_dummies(data, columns=["Opponent"])


In [456]:
data.head()

,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,...,Opponent_Portugal,Opponent_Russia,Opponent_Saudi Arabia,Opponent_Senegal,Opponent_Serbia,Opponent_Spain,Opponent_Sweden,Opponent_Switzerland,Opponent_Tunisia,Opponent_Uruguay
0,5,40,13,7,3,3,6,3,11,0,...,0,0,1,0,0,0,0,0,0,0
1,0,60,6,0,3,3,2,1,25,2,...,0,1,0,0,0,0,0,0,0,0
2,0,43,8,3,3,2,0,1,7,3,...,0,0,0,0,0,0,0,0,0,1
3,1,57,14,4,6,4,5,1,13,3,...,0,0,0,0,0,0,0,0,0,0
4,0,64,13,3,6,4,5,0,14,2,...,0,0,0,0,0,0,0,0,0,0


In [457]:
y = data["Man of the Match"]
x= data.drop("Man of the Match",axis=1)

In [458]:
data.columns

Index(['Goal Scored', 'Ball Possession %', 'Attempts', 'On-Target',
       'Off-Target', 'Blocked', 'Corners', 'Offsides', 'Free Kicks', 'Saves',
       'Pass Accuracy %', 'Passes', 'Distance Covered (Kms)',
       'Fouls Committed', 'Yellow Card', 'Yellow & Red', 'Red',
       'Man of the Match', '1st Goal', 'Round', 'PSO', 'Goals in PSO',
       'Team_Argentina', 'Team_Australia', 'Team_Belgium', 'Team_Brazil',
       'Team_Colombia', 'Team_Costa Rica', 'Team_Croatia', 'Team_Denmark',
       'Team_Egypt', 'Team_England', 'Team_France', 'Team_Germany',
       'Team_Iceland', 'Team_Iran', 'Team_Japan', 'Team_Korea Republic',
       'Team_Mexico', 'Team_Morocco', 'Team_Nigeria', 'Team_Panama',
       'Team_Peru', 'Team_Poland', 'Team_Portugal', 'Team_Russia',
       'Team_Saudi Arabia', 'Team_Senegal', 'Team_Serbia', 'Team_Spain',
       'Team_Sweden', 'Team_Switzerland', 'Team_Tunisia', 'Team_Uruguay',
       'Opponent_Argentina', 'Opponent_Australia', 'Opponent_Belgium',
       'Oppon

In [459]:
x["Round"]

0      0
1      0
2      0
3      0
4      0
      ..
123    3
124    4
125    4
126    5
127    5
Name: Round, Length: 128, dtype: int64

In [460]:
scaler = RobustScaler()
x=pd.DataFrame(scaler.fit_transform(x),columns=x.columns)

In [461]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.7)

#Training

In [462]:
sk_model = MLPClassifier(hidden_layer_sizes=(32,32))
sk_model.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(32, 32))

In [463]:
inputs = tf.keras.Input(shape=(85,))
x = tf.keras.layers.Dense(128, activation="relu")(inputs)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)

tf_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [464]:
tf_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [465]:
tf_model.fit(
    x_train,
    y_train,
    validation_split=0.4,
    batch_size=32,
    epochs=12
)

Epoch 1/12
2/2 [==============================] - 2s 217ms/step - loss: 0.6970 - accuracy: 0.4717 - val_loss: 0.8072 - val_accuracy: 0.3611
Epoch 2/12
2/2 [==============================] - 0s 37ms/step - loss: 0.6372 - accuracy: 0.6226 - val_loss: 0.8174 - val_accuracy: 0.4167
Epoch 3/12
2/2 [==============================] - 0s 34ms/step - loss: 0.6026 - accuracy: 0.7547 - val_loss: 0.7903 - val_accuracy: 0.5000
Epoch 4/12
2/2 [==============================] - 0s 49ms/step - loss: 0.5695 - accuracy: 0.8113 - val_loss: 0.7544 - val_accuracy: 0.5000
Epoch 5/12
2/2 [==============================] - 0s 37ms/step - loss: 0.5375 - accuracy: 0.8868 - val_loss: 0.7427 - val_accuracy: 0.5000
Epoch 6/12
2/2 [==============================] - 0s 49ms/step - loss: 0.4935 - accuracy: 0.8868 - val_loss: 0.7831 - val_accuracy: 0.5000
Epoch 7/12
2/2 [==============================] - 0s 35ms/step - loss: 0.4530 - accuracy: 0.8868 - val_loss: 0.8442 - val_accuracy: 0.5000
Epoch 8/12
2/2 [==========

#result

In [466]:
sk_score = sk_model.score(x_test, y_test)
tf_score = tf_model.evaluate(x_test, y_test, verbose=False)

In [467]:
print(f"   sklearn Model: {sk_score}")
print(f"TensorFlow Model: {tf_score[1]}")

   sklearn Model: 0.5641025641025641
TensorFlow Model: 0.6666666865348816
